In [1]:
import os

In [2]:
%pwd

'e:\\Kidney project\\Kidney-disease-classification-with-mlflow-dvc\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Kidney project\\Kidney-disease-classification-with-mlflow-dvc'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from KidneyClassification.constants import *
from KidneyClassification.utils.common import read_yaml, create_directories
import tensorflow as tf
import tensorflow as tf

tf.compat.v1.enable_eager_execution()
tf.config.run_functions_eagerly(True)


[2025-11-02 23:03:04,412: WARNING: module_wrapper: From C:\Users\Asus\AppData\Local\Temp\ipykernel_8152\1310436096.py:6: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.
]


In [7]:
tf.executing_eagerly()


True

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,        
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)     

        create_directories([self.config.artifacts_root])
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Kidney-CT-Scan-Images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import time


In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

        # ✅ FIX: Recompile optimizer after loading the model
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )


    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-11-02 23:03:16,056: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-02 23:03:16,058: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-02 23:03:16,059: INFO: common: created directory at: artifacts]
[2025-11-02 23:03:16,060: INFO: common: created directory at: artifacts\training]
[2025-11-02 23:03:16,285: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 1071 images belonging to 2 classes.
Found 4289 images belonging to 2 classes.


e:\New folder\envs\kidney\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


e:\New folder\envs\kidney\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


268/268 ━━━━━━━━━━━━━━━━━━━━ 1253s 5s/step - accuracy: 0.6006 - loss: 12.1464 - val_accuracy: 0.6761 - val_loss: 1.9691
Epoch 2/20
  1/268 ━━━━━━━━━━━━━━━━━━━━ 19:21 4s/step - accuracy: 0.6250 - loss: 2.7503

e:\New folder\envs\kidney\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


268/268 ━━━━━━━━━━━━━━━━━━━━ 80s 283ms/step - accuracy: 0.6250 - loss: 2.7503 - val_accuracy: 0.6960 - val_loss: 13.9456
Epoch 3/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1227s 5s/step - accuracy: 0.6374 - loss: 8.7811 - val_accuracy: 0.3040 - val_loss: 12.7169
Epoch 4/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 80s 282ms/step - accuracy: 0.5000 - loss: 13.1920 - val_accuracy: 0.6960 - val_loss: 11.9136
Epoch 5/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1237s 5s/step - accuracy: 0.6940 - loss: 6.5724 - val_accuracy: 0.7964 - val_loss: 1.3431
Epoch 6/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 80s 283ms/step - accuracy: 0.6250 - loss: 2.9966 - val_accuracy: 0.7472 - val_loss: 2.8445
Epoch 7/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1259s 5s/step - accuracy: 0.7117 - loss: 5.5826 - val_accuracy: 0.7623 - val_loss: 2.0519
Epoch 8/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 80s 282ms/step - accuracy: 0.5625 - loss: 4.4252 - val_accuracy: 0.6998 - val_loss: 8.7026
Epoch 9/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1347s 5s/step - accuracy: 0.7605 - loss: 3.7861 - val